In [ ]:
import json
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import requests
import time

In [ ]:
driver = webdriver.Chrome('C:\\Users\\Joanna\\AppData\\Local\\Programs\\Python\\Python39\\venv\\Scripts\\chromedriver.exe')

In [ ]:
def urls_for_keyword(keyword, driver=driver):    
    driver.get('https://www.google.pl/search?q=' + keyword)
    webelements = driver.find_elements_by_xpath('//div[@class="rc"]/div/a')
    urls = []
    for elem in webelements:
        urls.append(elem.get_attribute('href'))
    return urls

In [ ]:
def urls_info(urls, driver=driver):
    headings1 = []
    titles = []
    bodies = []
    for url in urls:
        driver.get(url)
        titles.append(driver.title)
        bodies.append(driver.find_elements_by_tag_name('body')[0].text)
        headings1.append(driver.find_elements_by_tag_name('h1')[0].text)
            
    df = pd.DataFrame(
    {'url' : urls,
     'title': titles,
     'h1' : headings1,
     'content' : bodies
     })
    return df

In [ ]:
def collect_data(url, category, position):
    api_content = requests.get('https://pagespeedonline.googleapis.com/pagespeedonline/v5/runPagespeed?url={}&category={}'.format(url, category)).content
    content = json.loads(api_content)
#     print(content)
    
    values = ['id', 'score', 'title', 'description']
    records = []

    for elem in content['lighthouseResult']['categories'][category.replace('_','-')]['auditRefs']:
        record = [{'position' : position}, {'url' : url}, {'category' : category}]
#         record = []
        record.append({'id': elem['id']})
        record.append({'weight' : elem['weight']})
        try:
            group = elem['group']
        except KeyError:
            group = np.nan
        record.append({'group' : group})
        records.append(record)
        values = ['score', 'title', 'description']
        for key, value in content['lighthouseResult']['audits'].items():
            if key == elem['id']:
                for v in values:
                    record.append({v : value[v]})
                    
    cols = ['position', 'url', 'category', 'id', 'weight', 'group', 'score', 'title', 'description']
    rows = []
    for record in records:
        row = []
        for elem in record:
            row.append([elem[key] for key in elem][0])
        rows.append(row)

    df = pd.DataFrame(rows, columns=cols)
    return df

In [ ]:
start = time.time()

keywords = ['czerwona sukienka', 'czarna sukienka']
categories = ['performance', 'best_practices', 'seo']

urls_informations =[]
for keyword in keywords:
    urls = urls_for_keyword(keyword)
    urls_information = urls_info(urls)
    urls_informations.append(urls_information)
    urls_information.to_csv('urls_info_for_keyword_{}.csv'.format(keyword))
    
    dfs = []
    for i, url in enumerate(urls):
        for category in categories:
            while True:
                try:
                    df = collect_data(url, category, i+1)
                except KeyError:
                    time.sleep(100)
                    continue
                break
            dfs.append(df)
            df.to_csv('keyword_{}_position_{}_category_{}.csv'.format(keyword, i+1, category))

stop = time.time()
print(stop-start)

In [ ]:
dfs[0]